<img align="right" src="../../additional_data/banner_siegel.png" style="width:1100px;">

# Parallel processing with Dask

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Products used**: 
* **Prerequisites**:  Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_jupyter_introduction.ipynb)
    * The basic structure of the eo2cube [satellite datasets](02_eo2cube.ipynb)
    * How to browse through the available [products and measurements](03_products_and_measurements.ipynb) of the eo2cube datacube 
    * How to [load data from the eo2cube datacube](04_loading_data_and_basic_xarray.ipynb)
    * How the data is stored and structured in a [xarray](05_advanced_xarray.ipynb)
    * How to [visualize the data](06_plotting.ipynb)
    * How to do a [basic analysis of remote sensing data](07_basic_analysis.ipynb) in the eo2cube environment

## What is Dask?
[Dask](https://dask.org/)

## Description / Structure of this Notebook?


## Package import and datacube connection

First of all you start with importing all the packages that are needed for our analysis.<br>
Most of the packages have already been introduced in the previous notebooks. The newly introduced package here is Dask.

In [13]:
import datacube

# dask
import dask
from dask.distributed import Client

connecting to the datacube

In [2]:
dc = datacube.Datacube(app = '08_parallel_processing_with_dask', config = '/home/datacube/.datacube.conf')

## Setting up Dask.distributed

In [15]:
#not working
#client=Client()


/home/datacube/anaconda3/envs/jupyterhub/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41829 instead
  warnings.warn(


KeyboardInterrupt: 

## Loading the data using dask
In the following, the data is loaded.
As you remeber from the previous notebooks a "normal" command for loading data looks somewhat like this: <br>
```python
ds = dc.load(product = "s2_l2a_bavaria",
             measurements = ["blue", "green","red", "red_edge2", "nir", "narrow_nir"],
             longitude = [12.493, 12.509],
             latitude = [47.861, 47.868],
             time = ("2020-04-01", "2021-03-31"))
```
For loading data with dask, you just add the "dask_chunks"-parameter. <br>
This looks like this:

In [7]:
# Load Data
ds = dc.load(product = "s2_l2a_bavaria",
             measurements = ["blue", "green","red", "red_edge2", "nir", "narrow_nir"],
             longitude = [12.493, 12.509],
             latitude = [47.861, 47.868],
             time = ("2020-04-01", "2021-03-31"),
             dask_chunks={"time": 10, "x": 50, "y": 50})

To understand what we have done here, lets first look at the dataset we loaded.

In [8]:
ds

<xarray.Dataset>
Dimensions:      (time: 360, x: 124, y: 84)
Coordinates:
  * time         (time) datetime64[ns] 2020-04-01T10:27:13 ... 2021-03-29T10:...
  * y            (y) float64 5.308e+06 5.308e+06 ... 5.307e+06 5.307e+06
  * x            (x) float64 7.612e+05 7.612e+05 ... 7.624e+05 7.624e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 dask.array<chunksize=(10, 50, 50), meta=np.ndarray>
    green        (time, y, x) int16 dask.array<chunksize=(10, 50, 50), meta=np.ndarray>
    red          (time, y, x) int16 dask.array<chunksize=(10, 50, 50), meta=np.ndarray>
    red_edge2    (time, y, x) int16 dask.array<chunksize=(10, 50, 50), meta=np.ndarray>
    nir          (time, y, x) int16 dask.array<chunksize=(10, 50, 50), meta=np.ndarray>
    narrow_nir   (time, y, x) int16 dask.array<chunksize=(10, 50, 50), meta=np.ndarray>
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

You see that it consists of multiple dask.array objects but we cannot see any vlaues inside our data. <br>
This type of data is called "Lazy data" as it is not loaded properly but in a "lazy" way without data values.

## Dask Chunks
The parameter that was added to the dc.load() command is called "dask_chunks". It defines in how many parts our original dataset will be splitted. As described in the previous notebooks, normally the dc.load() command produces a xarray dataset(??).<br>
In our case, the dataset is split into smaller chunks. Since the data we are interested in is three dimensional, we also need to provide three dimensions for subdividing the data. <br>
The provided values have been {"time": 10, "x": 50, "y": 50}. Accordingly, the chunksize of the dask.arrays is (10,50,50). <br>
<br>
In the following we will visualize the lazy-loaded data for the red band to get an even better feeling about our type of data.


In [9]:
# visualizing the dask chunks
ds.red.data

dask.array<rechunk-merge, shape=(360, 84, 124), dtype=int16, chunksize=(10, 50, 50), chunktype=numpy.ndarray>

We see that our data has been devided into a total of 216 chunks, each having a size of 10 timesteps, 50 pixels in x-direction, and 50 pixels in y-direction. <br>
Looking at the memory size of the chunks compared to the complete array, the motivation for using dask becomes clear. Especially when working with large amounts of data, splitting the data into smaller chunks enables computations that would crash the Phenocube environment when calculated over the complete array at once.

## Loading the lazy data
Of course it is possible to convert lazy data into "normal" data (xarray ???). This can be done using the .load() command. <br>
The resulting data has the same format as data loaded without dask.

In [11]:
ds=ds.load()

KeyboardInterrupt: 

## Recommended next steps

To continue with the beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](01_jupyter_introduction.ipynb)
2. [eo2cube](02_eo2cube.ipynb)
3. [Products and Measurements](03_products_and_measurements.ipynb)
4. [Loading data and introduction to xarrays](04_loading_data_and_basic_xarray.ipynb)
5. [Advanced xarrays operations](05_advanced_xarray.ipynb)
6. [Plotting data](06_plotting.ipynb)
7. [Basic analysis of remote sensing data](07_basic_analysis.ipynb)
8. **Parallel processing with Dask (this notebook)**

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks!</font>

https://doi.org/10.26186/145234 <br>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** February 2021